## Testing OpenAi 

- https://platform.openai.com/docs/guides/fine-tuning/example-notebooks

In [3]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='openai-gpt')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\zachz\Documents\GitHub\modelTesting\venv\lib\site-packages\transformers\generation\utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[{'generated_text': "Hello, I'm a language model,'he said, when i was finished.'ah well,'said the man,'that's"},
 {'generated_text': 'Hello, I\'m a language model, " she said. \n she reached the bottom of the shaft and leaned a little further out. it was'},
 {'generated_text': 'Hello, I\'m a language model, " she laughed. " we call that a\'white girl.\'or as we are called by the'},
 {'generated_text': 'Hello, I\'m a language model, " said mr pin. " an\'the ones with the funny hats don\'t. " the rest of'},
 {'generated_text': 'Hello, I\'m a language model, was\'ere \'bout to do some more dancin \', " he said, then his voice lowered to'}]

### Finetuning Openai-gpt

In [5]:
from transformers import OpenAIGPTTokenizer, OpenAIGPTModel
import torch


In [6]:
# Load the pretrained Openai-gpt model and tokenizer
tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
model = OpenAIGPTModel.from_pretrained("openai-gpt")

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [7]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
last_hidden_states

tensor([[[ 0.4653,  0.0642,  0.5910,  ...,  0.1177, -0.0021, -1.2262],
         [-0.3697, -0.0957,  0.6613,  ..., -0.0344, -0.2164,  0.1205],
         [ 0.1700, -0.3252,  0.0407,  ...,  0.1589, -0.8057, -0.2830],
         [-0.3669, -0.0448,  0.8061,  ..., -0.0090, -0.0872, -0.5224],
         [-0.5047,  0.6522,  0.6932,  ...,  0.0811,  0.6475,  0.3190],
         [-0.2972,  0.0591,  1.2333,  ..., -0.7394, -0.2600,  0.0863]]],
       grad_fn=<ViewBackward0>)

#### Train the model

In [8]:
# Load your dataset
with open('./data/calregs.txt', 'r', encoding='utf-8') as f:
    data = f.read()

# Tokenize the dataset
encoded_data = tokenizer.encode(data)

# Create input-output pairs for training
seq_len = model.config.n_positions
input_seqs = []
label_seqs = []
for i in range(0, len(encoded_data) - seq_len, seq_len):
    input_seqs.append(encoded_data[i:i+seq_len])
    label_seqs.append(encoded_data[i+1:i+seq_len+1])

# Convert input-output pairs to PyTorch tensors
input_seqs = torch.tensor(input_seqs)
label_seqs = torch.tensor(label_seqs)

# Create a PyTorch DataLoader for batching the input-output pairs
batch_size = 4
data_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(input_seqs, label_seqs),
    batch_size=batch_size,
    shuffle=True
)

Token indices sequence length is longer than the specified maximum sequence length for this model (162804 > 512). Running this sequence through the model will result in indexing errors
